In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as const
import sys
sys.path.append('/home/beards/code/epic_calculations/epic_calculations')
import telescope_observation
reload(telescope_observation)  # Do this while I'm developing telescope_observation
from telescope_observation import TelescopeObservation

In [ ]:
dsa_layout = np.loadtxt('/home/beards/code/epic_calculations/layouts/dsa_5m_15km.txt')
ovro_lwa_layout = np.loadtxt('/home/beards/code/epic_calculations/layouts/lwa_5km.txt')
ngvla_core_layout = np.loadtxt('/home/beards/code/epic_calculations/layouts/ngvla_core.txt')
ngvla_sba_layout = np.loadtxt('/home/beards/code/epic_calculations/layouts/ngvla_sba.txt')
ngvla_compact_layout = np.concatenate([ngvla_core_layout, ngvla_sba_layout], axis=0)
ska_low_layout = np.loadtxt('/home/beards/code/epic_calculations/layouts/ska_low.txt')
mwa_compact_layout = np.loadtxt('/home/beards/code/epic_calculations/layouts/mwa_phase2_compact.txt')
mwa_extended_layout = np.loadtxt('/home/beards/code/epic_calculations/layouts/mwa_phase2_extended.txt')
mwa_iii_layout = np.concatenate([mwa_compact_layout, mwa_extended_layout])
hera_layout = np.loadtxt('/home/beards/code/epic_calculations/layouts/HERA350.txt')

# Open questions
- What integration time to use?
- How many channels should we assume for each telescope?
- Grid size - use FoV at lowest end of band? Center of band?
- Pick center frequency for tunable arrays (MWA)
- HERA layout

In [ ]:
Nchan = 1024
dsa = TelescopeObservation(layout=dsa_layout, Dant=5, f0=1.35e3, bandwidth=1.3e3,
                           Nchan=Nchan, integration=1e-3)
ovro = TelescopeObservation(layout=ovro_lwa_layout, Dant=1.5, f0=55, bandwidth=60.,
                            Nchan=Nchan, integration=1e-3)
ngvla = TelescopeObservation(layout=ngvla_compact_layout, Dant=18, f0=11.2e3, bandwidth=20e3,
                             Nchan=Nchan, integration=1e-3)
ska_low = TelescopeObservation(layout=ska_low_layout, Dant=35, f0=200, bandwidth=300.,
                               Nchan=Nchan, integration=1e-3)
ska_low_core = TelescopeObservation(layout=ska_low_layout, Darray=1e3, Dant=35, f0=200, bandwidth=300.,
                                    Nchan=Nchan, integration=1e-3)
ska_low_extreme_core = TelescopeObservation(Darray=1e3, Nant=131072. / 2., Dant=1., f0=200, bandwidth=300.,
                                            Nchan=Nchan, integration=1e-3)
mwa_ii_compact = TelescopeObservation(layout=mwa_compact_layout, Dant=4.5, f0=200, bandwidth=30.72,
                                      Nchan=Nchan, integration=1e-3)
mwa_ii_extended = TelescopeObservation(layout=mwa_extended_layout, Dant=4.5, f0=200, bandwidth=30.72,
                                       Nchan=Nchan, integration=1e-3)
mwa_iii = TelescopeObservation(layout=mwa_iii_layout, Dant=4.5, f0=200, bandwidth=30.72 * 2,
                                       Nchan=Nchan, integration=1e-3)
puma_light = TelescopeObservation(Darray=600., Nant=5e3, Dant=6., f0=650, bandwidth=900.,
                                  Nchan=Nchan, integration=1e-3)
puma_full = TelescopeObservation(Darray=1500., Nant=32e3, Dant=6., f0=650, bandwidth=900.,
                                 Nchan=Nchan, integration=1e-3)
hera = TelescopeObservation(layout=hera_layout, Dant=14., f0=150., bandwidth=200., Nchan=8192,
                            integration=10)

In [ ]:
obs_dict = {'DSA-2000': dsa, 'OVRO-LWA-2000': ovro, 'NGVLA': ngvla, 'SKA-Low': ska_low, 'SKA-Low Core': ska_low_core,
            'SKA-Low Extreme Core': ska_low_extreme_core, 'MWA-II Compact': mwa_ii_compact,
            'MWA-II Extended': mwa_ii_extended, 'MWA-III': mwa_iii, 'PUMA light': puma_light,
            'PUMA Full': puma_full, 'HERA': hera}
epic_flops = np.array([1e-9 * obs.vanilla_EPIC_stats(verbose=False) for obs in obs_dict.values()])
fx_flops = np.array([1e-9 * obs.FX_stats(verbose=False) for obs in obs_dict.values()])

In [ ]:
plt.figure()
plt.loglog(fx_flops, epic_flops, 'o')
plt.plot([np.min(epic_flops), np.max(fx_flops)], [np.min(epic_flops), np.max(fx_flops)], ':k')
for label, x, y in zip(obs_dict.keys(), fx_flops, epic_flops):
    plt.annotate(label, (x, y))
plt.xlabel('FX Compute (GFLOPS)')
plt.ylabel('EPIC Compute (GFLOPS)')